In [6]:
import numpy as np

#### Dataset
http://thinknook.com/twitter-sentiment-analysis-training-corpus-dataset-2012-09-22/

pandas read_csv gives error because there is a value in some row in 4th column and all other rows of this col are empty.

In [2]:
sent_csv = np.genfromtxt("datasets/sentiment.csv",delimiter=',',usecols=(1,3),skip_header=1,dtype='str')

In [3]:
train_x = sent_csv[:,1]
train_y = sent_csv[:,0]

In [5]:
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import json

In [5]:
max_words = 1000 # max_words = 3000 gives out of memory error on 34 GB system :-(
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_x)

In [6]:
dictionary = tokenizer.word_index

In [20]:
with open('datasets/dictionary.json','w') as dictionary_file:
    json.dump(dictionary,dictionary_file)

In [8]:
def convert_text_to_indices(text):
    return [dictionary[word] for word in kpt.text_to_word_sequence(text)]

In [9]:
allWordIndices = []

for text in train_x:
    indcies = convert_text_to_indices(text)
    allWordIndices.append(indcies)
    
allWordIndices = np.asarray(allWordIndices)

In [11]:
train_x = tokenizer.sequences_to_matrix(allWordIndices,mode='binary')
train_y = keras.utils.to_categorical(train_y)

In [12]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [13]:
model = Sequential()

model.add(Dense( 512, input_shape=(max_words,), activation='relu' ))
model.add( Dropout(0.5) )
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [14]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
history = model.fit(train_x, train_y, batch_size=32, epochs=3, validation_split=0.1, shuffle=True)

Train on 1420764 samples, validate on 157863 samples
Epoch 1/3
1420764/1420764 [==============================] - 194s - loss: 0.5067 - acc: 0.7547 - val_loss: 0.4886 - val_acc: 0.7679
Epoch 2/3
1420764/1420764 [==============================] - 194s - loss: 0.5031 - acc: 0.7578 - val_loss: 0.4812 - val_acc: 0.7715
Epoch 3/3
1420764/1420764 [==============================] - 194s - loss: 0.5008 - acc: 0.7599 - val_loss: 0.4780 - val_acc: 0.7702


In [21]:
model_json = model.to_json()
with open('datasets/model.json','w') as json_file:
    json_file.write(model_json)

In [22]:
model.save_weights('datasets/model.h5')